In [1]:
import os
import cv2
import numpy as np
from deepface import DeepFace
#from deepface.commons import functions

from time import time

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

# Definición de funciones 

In [2]:
def GetSVMPredictions(X_train, X_test, y_train, y_test):
    
    print("+++++++++++++++++++++++++++++++++++++++++++\n")
    print("SVM Normalization...")
    scaler = MinMaxScaler()
    train_X = scaler.fit_transform(X_train)
    test_X = scaler.transform(X_test)

    print("SVM training...")
    t0 = time()
    parameters = {'C': [1e3, 5e3, 1e4, 5e4, 1e5],
                  'gamma': [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.1],}
    # Grid serach across parameter range
    clf = GridSearchCV(
        SVC(kernel='rbf', class_weight='balanced'), parameters, cv=5
    )
    clf = clf.fit(train_X, y_train)
    print("done in %0.3fs" % (time() - t0))
    print("Best estimator found by grid search:")
    print(clf.best_estimator_)


    print("Predicting")
    t0 = time()
    y_pred = clf.predict(test_X)
    print("done in %0.3fs" % (time() - t0))
    
    return y_pred, y_test

def GetKNNPredictions(k, X_train, X_test, y_train, y_test):
    
    t0 = time()
    # k = 5 
    model_px = KNeighborsClassifier(n_neighbors = k) 

    # fdtraining of model 
    model_px.fit(X_train, y_train) 
    print("done in %0.3fs" % (time() - t0))

    print("Predicting...")
    t0 = time()
    y_pred=model_px.predict(X_test)

    print("done in %0.3fs" % (time() - t0))
    
  
    return y_pred, y_test

In [3]:
def calc_embs(X, batch_size=2): #No usada desde 2024 en el código
    norm_images = prewhiten(X)
    pd = []
    for start in range(0, len(norm_images), batch_size):
        pd.append(model.predict_on_batch(norm_images[start:start+batch_size]))     #https://github.com/serengil/deepface/issues/819           
    return l2_normalize(np.concatenate(pd))

def l2_normalize(x, axis=-1, epsilon=1e-10):
    output = x / np.sqrt(np.maximum(np.sum(np.square(x), axis=axis, keepdims=True), epsilon))
    return output

def prewhiten(x):
    if x.ndim == 4:
        axis = (1, 2, 3)
        size = x[0].size
    elif x.ndim == 3:
        axis = (0, 1, 2)
        size = x.size
    else:
        raise ValueError('Dimension should be 3 or 4')

    mean = np.mean(x, axis=axis, keepdims=True)
    std = np.std(x, axis=axis, keepdims=True)
    std_adj = np.maximum(std, 1.0/np.sqrt(size))
    y = (x - mean) / std_adj
    return y


def LoadDataset(folder, ext):
    # Contador de número de clases del conjunto
    nclasses = 0
    # Contador de muestras por clase
    nperclass = []
    # Etiqueta de cada clase (nombre de la subcarpeta)
    classlabels = []
    # Inicializa estructuras de datos y sus correpondientes etiquetas
    X = []
    Y = []

    preprocessing = 0

    # Asume que en la ruta indicada hay una subcarpeta por clase
    for class_name in os.listdir(folder):
        # Cada subcarpeta implica una clase más
        nclasses += 1
        # Inicialmente esta clase no tiene muestras
        nsamples = 1    

        # Compone la ruta
        class_folder = os.path.join(folder, class_name)
        for file_name in os.listdir(class_folder):
            # Asume imágenes en formato ext
            if file_name.endswith(ext):
                # Lee la imagen
                image = cv2.imread (os.path.join(class_folder, file_name))  

                # Obtiene embeddings
                img1 = cv2.resize(image, dim, interpolation = cv2.INTER_AREA)

                # Hasta 2023 usaba
                # Get embeddings after preprocessing
                #if preprocessing == 1:
                #    img_embedding = calc_embs(np.array([img1]))
                #else:
                #    img_embedding = model.predict(img1[None,...])

                #X.append(img_embedding[0])
                
                #Desde 2024
                embedding_objs = DeepFace.represent(img_path = img1,model_name  = "Facenet", enforce_detection = False)   
                img_embedding = embedding_objs[0]["embedding"]             

                X.append(img_embedding)

                # Añade etiqueta numérica de la muestra
                Y.append(nclasses-1)

                #Incrementa el número de muestras
                nsamples += 1

        nperclass.append(nsamples)
        classlabels.append(class_name)

    #Convierte a numpy array X e Y
    X = np.array(X,dtype='float32')
    Y = np.array(Y,dtype='float64')

    # Muestra datos del conjunto leído
    # Depuración
    print("Features")
    print(X.shape)
    print(Y.shape)
    # Obtiene número de muestras y características
    n_samples , n_features = X.shape
    # Obtiene nombres de las clases
    class_names = np.array(classlabels)
    n_classes = class_names.shape[0]
    
    return X, Y, n_samples, n_features, n_classes, classlabels, nperclass, class_names



# Carga conjuntos de datos

Se proporciona la carpeta, a través de la variable folder, donde cada subcarpeta se corresponde con una clase.
Cada clase contiene muestras en forma de imágenes jpg, todas del mismo tamaño. Obtiene embeddings Facenet

In [4]:
# Modelos disponibles ['VGG-Face', 'Facenet', 'OpenFace', 'DeepFace', 'DeepID', 'Dlib']
model = DeepFace.build_model("Facenet")
dim = model.input_shape

#MODIFICAR INDICANDO RUTA EN TU EQUIPO TRAS DESCAGAR DATOS DEL CAMPUS. EVITAR TILDES
folder = "./DatabaseGender59x65/DatabaseGender59x65" #RUTA EJEMPLO, MODIFICAR

print('Loading dataset')
X, Y, nsamples, class_name, nperclass, classlabels, width, height = LoadDataset(folder,'.jpg')

#Convierte a numpy array X e Y
X = np.array(X,dtype='float32')
Y = np.array(Y,dtype='float64')

# Obtiene número de muestras y características
n_samples , n_features = X.shape
# Obtiene nombres de las clases
class_names = np.array(classlabels)
n_classes = class_names.shape[0]

print("Dataset info:")
print("# samples: %d" % n_samples)
print("# features: %d" % n_features)
print("# classes: %d" % n_classes)
print("classes %s" % classlabels)
print("samples per class %s" % str(nperclass)[1:-1] )




25-11-13 12:26:04 - 🔗 facenet_weights.h5 will be downloaded from https://github.com/serengil/deepface_models/releases/download/v1.0/facenet_weights.h5 to C:\Users\andsa\.deepface\weights\facenet_weights.h5...


Downloading...
From: https://github.com/serengil/deepface_models/releases/download/v1.0/facenet_weights.h5
To: C:\Users\andsa\.deepface\weights\facenet_weights.h5
100%|██████████| 92.2M/92.2M [00:14<00:00, 6.48MB/s]


Loading dataset
Features
(1184, 128)
(1184,)
Dataset info:
# samples: 1184
# features: 128
# classes: 2
classes ['Female', 'Male']
samples per class 


# Diseña conjunto experimental k-fold

Divide los datos k veces en conjunto de entrenamiento y test

In [5]:
# StratifiedKFold
# Define el número de subconjuntos a considerar
kfold = 5
skf = StratifiedKFold(n_splits=kfold, random_state=4, shuffle=True)
#Distribución de muestras por fold
fold = 1
for train_index, test_index in skf.split(X, Y):
    #print("TRAIN:", train_index, "TEST:", test_index)
    print("Fold %d" % fold)
    print("# samples in training set %d" % train_index.shape[0])
    print("# samples in test set %d" % test_index.shape[0])
    fold += 1

Fold 1
# samples in training set 947
# samples in test set 237
Fold 2
# samples in training set 947
# samples in test set 237
Fold 3
# samples in training set 947
# samples in test set 237
Fold 4
# samples in training set 947
# samples in test set 237
Fold 5
# samples in training set 948
# samples in test set 236


# Lanza experimento

In [6]:
# Embeddings
precs_facenet_svm, recs_facenet_svm = [], []
precs_facenet_knn, recs_facenet_knn = [], []


# Recorre folds
fold = 1
while fold <= kfold:
    accs, precs, recs = [], [], []
    for train_index, test_index in skf.split(X, Y):
        #print("TRAIN:", train_index, "TEST:", test_index)
        print("***\nFold %d" % fold)
        #División de muestras de entreno y test
        X_train, X_test = X[train_index], X[test_index]
        #Etiquetas de las muestras
        y_train, y_test = Y[train_index], Y[test_index]


        #Facenet+KNN
        y_pred, y_test = GetKNNPredictions(11, X_train, X_test,y_train, y_test)
        print("\nFacenet+KNN Metrics")
        precs_facenet_knn.append(precision_score(y_test, y_pred, average='weighted'))
        recs_facenet_knn.append(recall_score(y_test, y_pred, average='weighted'))
        print(classification_report(y_test, y_pred, target_names=class_names))
        print(confusion_matrix(y_test, y_pred, labels=range(n_classes)))      
            
        #Facenet+SVM
        y_pred, y_test = GetSVMPredictions(X_train, X_test,y_train, y_test)
        print("\nFacenet+SVM Metrics")
        precs_facenet_svm.append(precision_score(y_test, y_pred, average='weighted'))
        recs_facenet_svm.append(recall_score(y_test, y_pred, average='weighted'))
        print(classification_report(y_test, y_pred, target_names=class_names))
        print(confusion_matrix(y_test, y_pred, labels=range(n_classes)))        
    

    fold += 1

print("Facenet+KNN Mean Precision:  %0.3f, Mean Recall:  %0.3f" % ( np.mean(precs_facenet_knn) , np.mean(recs_facenet_knn) ))
print("Facenet+SVM Mean Precision:  %0.3f, Mean Recall:  %0.3f" % ( np.mean(precs_facenet_svm) , np.mean(recs_facenet_svm) ))


***
Fold 1
done in 0.001s
Predicting...
done in 2.194s

Facenet+KNN Metrics
              precision    recall  f1-score   support

      Female       0.71      0.96      0.82       117
        Male       0.94      0.62      0.75       120

    accuracy                           0.79       237
   macro avg       0.83      0.79      0.78       237
weighted avg       0.83      0.79      0.78       237

[[112   5]
 [ 45  75]]
+++++++++++++++++++++++++++++++++++++++++++

SVM Normalization...
SVM training...
done in 5.394s
Best estimator found by grid search:
SVC(C=1000.0, class_weight='balanced', gamma=0.01)
Predicting
done in 0.006s

Facenet+SVM Metrics
              precision    recall  f1-score   support

      Female       0.88      0.91      0.89       117
        Male       0.91      0.88      0.89       120

    accuracy                           0.89       237
   macro avg       0.89      0.89      0.89       237
weighted avg       0.89      0.89      0.89       237

[[106  11]
 [ 1